# Store Sales Average Model
---
Notebook ini merupakan studi mengenai hubungan antara Jenis Bangunan, Kelas Bangunan, Lokasi Kabupaten/Kota dan Luas Area Toko dalam men-generate penjualan per bulan di PT Prestasi Retail Innovation.
- Jenis Bangunan: Merupakan satu dari tiga tipe bangunan diantaranya Mall (M), Ruko (R) dan Bangunan Sendiri (S)
- Kelas Bangunan: Merupakan kelas dari tipe bangunan tersebut dalam men-generate sales. Misalkan Mall Grand Indonesia adalah Mall Kelas 1 (M1) sedangkan Gandaria City adalah Mall Kelas 4 (M4). Berikut adalah Kelas Bangunan yang ada.
  - Mall Kelas 1 (M1)
  - Mall Kelas 2 (M2)
  - Mall Kelas 3 (M3)
  - Mall Kelas 4 (M4)
  - Mall Kelas 5 (M5)
  - Ruko Kelas 1 (R1)
  - Ruko Kelas 2 (R2)
  - Ruko Kelas 3 (R3)
  - Ruko Kelas 4 (R4)
  - Ruko Kelas 5 (R5)
  - Bangunan Sendiri Kelas 1 (S1)
  - Bangunan Sendiri Kelas 2 (S2)
  - Bangunan Sendiri Kelas 3 (S3)
  - Bangunan Sendiri Kelas 4 (S4)
  - Bangunan Sendiri Kelas 5 (S5)
- Lokasi Kabupaten/Kota adalah lokasi geografis dari toko
- Luas Area Toko adalah luas meter persegi dari toko
---
Studi ini akan menggunakan Regresi Linear dalam merumuskan nilai Average Sales ($y$) yang dipengaruhi oleh variabel - variabel independen lainnya seperti Jenis Bangunan $x{_1}$, Kelas Bangunan $x{_2}$, Lokasi Kabupaten/Kota ($x{_3}$) dan Luas Area Toko ($x{_4}$). Berikut adalah formulasi Regresi Linear untuk permasalahan tersebut:  
  
$
y = ax{_1} + bx{_2} + cx{_3} + dx{_4} + e
$  
  
Dimana:  
$y$ = Prediksi Average Sales  
$a$ = Koefisien variabel independen $x{_1}$  
$b$ = Koefisien variabel independen $x{_2}$  
$c$ = Koefisien variabel independen $x{_3}$  
$d$ = Koefisien variabel independen $x{_4}$  
$e$ = Bias dari Regresi Linear 
$x{_1}$ = Variabel independen mewakili Jenis Bangunan  
$x{_2}$ = Variabel independen mewakili Kelas Bangunan  
$x{_3}$ = Variabel independen mewakili Lokasi Kabupaten/Kota  
$x{_4}$ = Variabel independen mewakili Luas Area Toko  

### Import Modul

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Versi Tensorflow

In [ ]:
print(tf.__version__)
print(tf.config.list_physical_devices('CPU'))

## Eksplorasi Data
---

### Atribut Dasar Toko
Berikut adalah beberapa data atribut dasar toko yang saat ini dimiliki.

In [ ]:
data_toko = pd.read_excel("PRI - Store Renov  Rent.xlsx", sheet_name=0, header=0)
print(data_toko.to_string())

#### Luas Area Toko (sqm)

In [ ]:
data_luas = data_toko["sqm"]
hitung, bin = np.histogram(data_luas)
print(hitung, bin)

In [ ]:
plt.hist(bin[:-1], weights=hitung)
plt.title("Persebaran Toko berdasar SQM")
plt.xlabel("Square Meters")
plt.ylabel("Jumlah Toko")
plt.show()

Berdasarkan luasnya (sqm), kita dapat melihat pada fungsi `histogram` di atas bahwa distribusi persebaran luas toko cukup normal dengan 19 Toko jatuh ke dalam kategori luas (sqm) diantara 80.874 $m{^2}$ sampai 119.748 $m{^2}$ dan masing - masing 10 toko jatuh ke dalam kategori luas (sqm) 42 $m{^2}$ sampai 80.874 $m{^2}$ dan 119.748 $m{^2}$ sampai 158.622 $m{^2}$ dengan pengecualian 1 Toko yang jatuh ke dalam kategori luas (sqm) 158.622 $m{^2}$ sampai 197.496 $m{^2}$ serta 1 Toko yang menjadi *outlier* dengan luas yang jatuh ke dalam kategori luas (sqm) 391.866 $m{^2}$ sampai 430.74 $m{^2}$

#### Penjualan
---
Mengingat bahwa data *historical* yang dimiliki terbatas dari tahun 2018 sampai dengan November 2022, serta mengingat bahwa kita mengalami periode pandemi CoV-19 selama lebih dari 1 tahun, maka penulis merasa perlu untuk melakukan separasi data penjualan per bulan menggunakan flag `Pandemic`.
  
Berikut adalah sepenggal data penjualan *historical* per toko dari tahun 2018 sampai dengan November 2022 (40 baris data awal).

In [ ]:
data_penjualan = pd.read_excel("PRI - Store Renov  Rent.xlsx", sheet_name="Sales", header=0)
print(data_penjualan.head(40))

Berikut adalah grouping data penjualan *historical* yang disimpan dalam variabel `data_penjualan_by_month`.

In [ ]:
data_penjualan_by_month = data_penjualan.groupby(["EOM"], ).sum(numeric_only=True)
print(data_penjualan_by_month.to_string())

Untuk melihat trend pergerakan penjualan dalam kurun waktu ini, kita akan menggunakan `Simple Moving Average` yang akan menghitung rerata secara bergulung untuk interval waktu ke belakang (contoh: Moving Average 3 Bulan untuk Mei 2021 adalah rata - rata penjualan yang merupakan rata - rata dari penjualan di bulan Maret 2021, April 2021 dan Mei 2021).  
Namun satu `Moving Average` saja tidak dapat menggambarkan sebuah trend, karena itu kita juga akan menggunakan tambahan 2 interval `Moving Average` lainnya untuk menggambarkan trend pada jangka pendek, jangka menengah dan jangka panjang.  
Dalam studi ini kita akan menggunakan 3 `Simple Moving Average` yaitu:
* `MA3`: `Moving Average` dengan jendela periode 3 bulan ke belakang (Jangka Pendek)
* `MA6`: `Moving Average` dengan jendela periode 6 bulan ke belakang (Jangka Menengah)
* `MA12`: `Moving Average` dengan jendela periode 12 bulan atau 1 tahun ke belakang (Jangka Panjang)  
Berikut adalah `data_penjualan_by_month` dengan penambahan kolom `MA3`, `MA6` dan `MA12` yang didapat dengan menggunakan fungsi `rolling()` dari `pd.DataFrame` yang dirata-ratakan dengan fungsi `mean()`.

In [ ]:
data_penjualan_by_month["MA3"] = data_penjualan_by_month["Sales"].rolling(3).mean()
data_penjualan_by_month["MA6"] = data_penjualan_by_month["Sales"].rolling(6).mean()
data_penjualan_by_month["MA12"] = data_penjualan_by_month["Sales"].rolling(12).mean()
print(data_penjualan_by_month.to_string())

Berikut adalah grafik penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
plt.plot(data_penjualan_by_month["Sales"])
plt.show()

Berikut adalah grafik `Moving Average` untuk data penjualan *historical* dari tahun 2018 sampai dengan November 2022.

In [ ]:
# plt.plot(data_penjualan_by_month["MA3"], color="green", label="MA3")
# plt.plot(data_penjualan_by_month["MA6"], color="orange", label="MA6")
# plt.plot(data_penjualan_by_month["MA12"], color="red", label="MA12")
# plt.show()
fig, ax = plt.subplots()
# Plot Moving Average
ax.plot(data_penjualan_by_month["MA3"], color="green", label="MA3")
ax.plot(data_penjualan_by_month["MA6"], color="orange", label="MA6")
ax.plot(data_penjualan_by_month["MA12"], color="red", label="MA12")
# Region Section Pandemic
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values >= np.datetime64('2020-02-29')) & (data_penjualan_by_month.index.values <= np.datetime64('2021-10-31'))), color="red", alpha=0.2)
# Region Section Recovery
ax.fill_between(data_penjualan_by_month.index.values, 0, 28000000000, where=((data_penjualan_by_month.index.values >= np.datetime64('2021-11-01')) & (data_penjualan_by_month.index.values <= np.datetime64('2022-12-31'))), color="green", alpha=0.2)
plt.show()